In [3]:
import os
%pwd

'/home/ubuntu/africlimateai/rainfall-prediction/research'

In [10]:
os.chdir("../")
%pwd

'/home/ubuntu/africlimateai/rainfall-prediction'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [12]:


from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH,
                 ):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)
                create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )
        return data_ingestion_config

In [14]:
import pandas as pd
from mlProject.utils.common import get_size
from mlProject import logger

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            data = pd.read_csv(self.config.source_URL)
            data.to_csv(self.config.local_data_file, index=False)
            logger.info(f"data download with following info: shape:{data.shape}")
        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))} bytes")
            
        

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
except Exception as e:
    raise e

[2025-09-23 09:32:37,619: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-23 09:32:37,620: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-23 09:32:37,622: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-23 09:32:37,624: INFO: common: created directory at: artifacts]
[2025-09-23 09:32:37,625: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-23 09:32:37,763: INFO: 1308463248: data download with following info: shape:(267, 13)]
